<a href="https://colab.research.google.com/github/Madeleinekohbc/datagovsg-datasets/blob/master/DataGovSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This Jupyter notebook makes it easy to :

1. Get the dataset and column metadata programmatically
2. Load CSV files automatically into a pandas dataframe so you can do the fun explorations

# Setup
1. Paste the dataset ID you copied into the cell below
2. Run All Cells (click `Runtime` -> `Run All`)

In [ ]:
DATASET_ID = "PASTE_DATASET_ID_HERE" # e.g. "d_69b3380ad7e51aff3a7dcc84eba52b8a"
API_KEY = "PASTE_API_KEY_HERE" #e.g. "v2:a7ae10..."

## Dataset and Column Metadata

In [1]:
import json
import requests

s = requests.Session()
s.headers.update({'referer': 'https://colab.research.google.com'})
if API_KEY and API_KEY != "PASTE_API_KEY_HERE":
    s.headers['x-api-key'] = API_KEY
s.headers.update(s.headers)
base_url = "https://api-production.data.gov.sg"
url = base_url + f"/v2/public/api/datasets/{DATASET_ID}/metadata"
print(url)
response = s.get(url)
data = response.json()['data']
columnMetadata = data.pop('columnMetadata', None)

print("Dataset Metadata:")
print(json.dumps(data, indent=2))

print("\nColumns:\n", list(columnMetadata['map'].values()))


NameError: name 'API_KEY' is not defined

## Download File

In [2]:
import time
import pandas as pd

def download_file(DATASET_ID, API_KEY=None):

  headers = {"Content-Type": "application/json"}
  if API_KEY:
      headers["x-api-key"] = API_KEY
  # initiate download
  initiate_download_response = s.get(
      f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download",
      headers=headers,
      json={}
  )
  print(initiate_download_response.json()['data']['message'])

  # poll download
  MAX_POLLS = 5
  for i in range(MAX_POLLS):
    poll_download_response = s.get(
        f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download",
        headers=headers,
        json={}
    )
    print("Poll download response:", poll_download_response.json())
    if "url" in poll_download_response.json()['data']:
      print(poll_download_response.json()['data']['url'])
      DOWNLOAD_URL = poll_download_response.json()['data']['url']
      df = pd.read_csv(DOWNLOAD_URL)

      display(df.head())
      print("\nDataframe loaded!")
      return df
    if i == MAX_POLLS - 1:
      print(f"{i+1}/{MAX_POLLS}: No result found, possible error with dataset, please try again or let us know at https://go.gov.sg/datagov-supportform\n")
    else:
      print(f"{i+1}/{MAX_POLLS}: No result yet, continuing to poll\n")
    time.sleep(3)

df = download_file(DATASET_ID)


NameError: name 'DATASET_ID' is not defined

In [ ]:
df.describe()

,bidding_no,quota,premium
count,1835.000000,1835.000000,1835.000000
mean,1.498638,565.764578,50516.278474
std,0.500134,420.596173,32259.684933
min,1.000000,43.000000,852.000000
25%,1.000000,292.000000,29085.000000
50%,1.000000,434.000000,49012.000000
75%,2.000000,659.500000,72051.500000
max,2.000000,2272.000000,158004.000000
